<a href="https://colab.research.google.com/github/Elwing-Chou/tibame0410/blob/main/tibame_0428_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request as req
import bs4 as bs
url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    print(link_href)
    # enter
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())
    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"})
    p_brand_text = p_brand.text.strip()
    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"})
    p_name_text = p_name.text.strip()
    # 滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"
            break
    # 評分
    p_rating_all = p_html.find("div", {"class":"c-ratingIcon__flex"})
    if not p_rating_all == None:
        p_rating = p_rating_all.find("div", {"class":"c-ratingIcon__textNumber"})
        p_rating_count = p_rating_all.find("a")
        p_rating_text = p_rating.text.strip()
        p_rating_count_text = p_rating_count.text.strip().replace("則評價", "")
    else:
        p_rating_text = "-"
        p_rating_count_text = "-"

    # 原價/價格
    p_pricebox = p_html.find("div", {"class":"o-prodPrice__priceBox"})
    p_price_now = p_pricebox.find("div", {"class":"o-prodPrice__price"})
    p_price_now_text = p_price_now.text.strip()
    p_price_origin = p_pricebox.find("div", {"class":"o-prodPrice__originalPrice"})
    if p_price_origin == None:
        p_price_origin_text = "-"
    else:
        p_price_origin_text = p_price_origin.text.strip()
    print("原價:", p_price_origin_text)
    print("目前售價:", p_price_now_text)
    print("評分:", p_rating_text)
    print("幾則:", p_rating_count_text)
    print("品牌:", p_brand_text)
    print("商品:", p_name_text)
    print("滿額贈:", p_full_prize)

https://24h.pchome.com.tw/prod/DMAH47-A900I9MK8
原價: $6,280
目前售價: $5,680
評分: -
幾則: -
品牌: sOlac
商品: So-Wrap 7 in 1 多功能專業造型吹風機(SR-1000G)
滿額贈: N
https://24h.pchome.com.tw/prod/DDBH8B-A900GK0TK
原價: -
目前售價: $1,900
評分: 5.0
幾則: 8
品牌: AVEDA
商品: 蘊活淨瑕頭皮調理液 100ml (鎮靜紓緩頭皮並祛除頭氣)
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAH7M-A900HPUFB
原價: $1,290
目前售價: $1,290
評分: -
幾則: -
品牌: acerpure 宏碁
商品: Acerpure beauty 雙效負離子吹風機 HD364-10W
滿額贈: N
https://24h.pchome.com.tw/prod/DDDU04-A900HHN7T
原價: -
目前售價: $5,400
評分: 4.5
幾則: 2
品牌: AVEDA
商品: 蘊活煥欣洗髮精 1000ml
滿額贈: Y
https://24h.pchome.com.tw/prod/DDDU04-1900I79JH
原價: -
目前售價: $1,650
評分: -
幾則: -
品牌: AVEDA
商品: 蘊活淨瑕養髮組
滿額贈: Y
https://24h.pchome.com.tw/prod/DMAHBY-A900I0K1F
原價: $1,295
目前售價: $1,295
評分: 4.9
幾則: 80
品牌: Xiaomi 小米
商品: 負離子高速吹風機
滿額贈: N
https://24h.pchome.com.tw/prod/DMAHCW-A900IGQBP
原價: $4,480
目前售價: $2,780
評分: 5.0
幾則: 4
品牌: KINYO
商品: 全球電壓無刷吹風機 KH-9660
滿額贈: N
https://24h.pchome.com.tw/prod/DMAH4C-A900I7YVV
原價: $2,990
目前售價: $2,799
評分: 4.9
幾則: 53
品牌: Panasonic 國際牌
商品: 高

In [ ]:
# 名字
# 原價/目前售價
# 滿額贈是否有(Y/N)
# 進去以後
# 評價幾分/幾則
# 介紹
# 圖片的網址